Name: __Arun Singh__

Email Id : erarungwl2013@gmail.com

Contact Number : +91 7430066111

#### Why I chose this problem?

I have never done this kind of stuff, which exites me. This is literally the first model that I am working on the related to image processing with deep learning.

#### What I had in my mind to solve the challenge?
As the given research paper regarding making meta learning to be more efficient, I understand the problem is the number of iterations that we need to do, for making a model do multiple things, even with the single data point. That is also the case in the N-way one shot learning classification to classify images in the classes, with a single image of them for training the model.
Since I don't have any experience, I stick to the paper, and I was trying to understand the relations between the classification task and the meta learning techniques. The given paper describes that, to learn many tasks and perform them efficiently using a single model, we can train the model initially and then fine tune with a single pass of a single task specific data point.
What I think is that more efficient approach of the one shot learning classification would be to train the model on the few sets of classes and then we can fine tune the model whenever there is something new comes in. So comparing the two different techniques, it is safe to assume that the task in meta-learning task is the classifying the image to be as its own class. Then the finding different outputs in a specific task would be same as generating the high similarity score for the items of the same class(same Task).

To analyse that the above mentioned theory works in a positive way, I was hoping to create a baseline with a simple or the current state of the art methods.

Keeping in mind that and my in-experience in this side of deep learning, I though it was better to stick with the simple siamese model for the startes. SO I started with creating a base line with a simple siamese network used for one shot learning as follows:

In [1]:
# importing necessary packages
import os
import numpy as np
import imageio
import pickle
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform, RandomNormal
from keras.models import Model

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
# from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
# data_loader.py handles the data reading and generating a simple python objects containing data
from data_loader import DataLoader

In [3]:
# batch_generator.py has the modules for the managing bactch which is genrating train and test batches for process
from batch_generator import BatchManager

In [15]:
# It has the model and it's training functions
from siamese import *

loading data

In [ ]:
data_path = './data'
loader = DataLoader(data_path)
loader.load_data()
train_X, train_y, train_alphabets_to_start_end = loader.training_data
test_X, test_y, test_alphabets_to_start_end = loader.evaluation_data
loader.store_data(path='.')

## Initializing model

In [5]:
root = '.'
data_path = os.path.join(root, 'data')

In [8]:
model_path = './weights/'

In [9]:
# initializ model
model = get_siamese_model((105, 105, 1))
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_1[1][0]         

/home/arun/miniconda3/envs/gpu/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


### Loading data pickle file

In [10]:
# load training data and evaluation data
with open(os.path.join(root, 'training_data.pkl'), 'rb') as readFile:
    training_data = pickle.load(readFile)

with open(os.path.join(root, 'evaluation_data.pkl'), 'rb') as readFile:
    evaluation_data = pickle.load(readFile)

In [11]:
training_manager = BatchManager(training_data)
evaluating_manager = BatchManager(evaluation_data)

#### Training Parameters

In [12]:
evaluation_frequency = 10
batch_size = 30
n_iter = 100
N = 20
total_number_of_validation_task = 100
best = -1

## Training

In [13]:
import time
start = time.time()

for i in range(1, n_iter+1):
    (inputs, targets) = training_manager.get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluation_frequency == 0:
        print('[{}] Time for {} iterations: {} mins'.format(i, i, (time.time() - start)/60))
        print('train loss: {}'.format(loss))
        accuracy = test_model(model,evaluating_manager, N, total_number_of_validation_task, batch_size=20, verbose=False)
        model.save_weights(os.path.join(root, 'weights.{}.h5'.format(i)))
        if accuracy >= best:
            print('current best: {}, previous best: {}'.format(accuracy, best))
            best = accuracy
        

[10] Time for 10 iterations: 1.290677539507548 mins
train loss: 6.0911030769348145
Accuracy: 25.0%  - 20 way one shot learning
current best: 25.0, previous best: -1
[20] Time for 20 iterations: 4.111621181170146 mins
train loss: 5.705084323883057
Accuracy: 35.0%  - 20 way one shot learning
current best: 35.0, previous best: 25.0
[30] Time for 30 iterations: 7.093730219205221 mins
train loss: 5.333745956420898
Accuracy: 37.0%  - 20 way one shot learning
current best: 37.0, previous best: 35.0
[40] Time for 40 iterations: 9.888330284754435 mins
train loss: 4.982854843139648
Accuracy: 34.0%  - 20 way one shot learning
[50] Time for 50 iterations: 12.428595920403799 mins
train loss: 4.653891086578369
Accuracy: 30.0%  - 20 way one shot learning
[60] Time for 60 iterations: 15.261238209406535 mins
train loss: 4.346142768859863
Accuracy: 35.0%  - 20 way one shot learning
[70] Time for 70 iterations: 17.516708755493163 mins
train loss: 4.060657978057861
Accuracy: 32.0%  - 20 way one shot learn

### Evaluating

In [18]:
test_model(model, evaluating_manager, 20, 30, 20, verbose=True)

Evaluating model on random vevaluation set of batch_size 20, in 20 way classification
Evaluating model on validation set with random 20 way one-shot learning tasks...
[evaluating] : Accuracy: 0.0%  - 20 way one shot learning
Evaluating model on random vevaluation set of batch_size 20, in 20 way classification
Evaluating model on validation set with random 20 way one-shot learning tasks...
[evaluating] : Accuracy: 50.0%  - 20 way one shot learning
Evaluating model on random vevaluation set of batch_size 20, in 20 way classification
Evaluating model on validation set with random 20 way one-shot learning tasks...
[evaluating] : Accuracy: 33.333333333333336%  - 20 way one shot learning
Evaluating model on random vevaluation set of batch_size 20, in 20 way classification
Evaluating model on validation set with random 20 way one-shot learning tasks...
[evaluating] : Accuracy: 25.0%  - 20 way one shot learning
Evaluating model on random vevaluation set of batch_size 20, in 20 way classificati

16.666666666666668

### Becasue of the unavailability of the GPU or high end CPU servers, I couldn't proceed further. I started just a week ago, So I wasn't manage to do much about it, spend days understanding and then creating this much...

If I may, I would still like to be considered as a candidate, and I know I would be able to deliver much more than this if I will be given the opportunity.
I am strogly motivated by my interests in the field and I am willing to put my time and efforts to the project, and learning that would be require exploring much more where my interest lies